In [1]:
import os
os.chdir('../..')


In [2]:
from classes.helpers import *
import numpy as np
import json
import pandas as pd
import scipy.stats as ss
from sklearn import linear_model
import ast
import scipy.stats as ss

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

data_folder = '../data/'
min_nbr_rats = 5

# Load the DF with the ranks

In [3]:
df = pd.read_csv(data_folder + 'tmp/time_series_{}_ranks.csv'.format(min_nbr_rats), header=[0,1])

# Transform strings into arrays
for i in df.index:
    for key1 in ['ba', 'rb']:
        for key2 in ['dates', 'ratings', 'z_scores', 'ranks']:
            arr = ast.literal_eval(df.loc[i][key1][key2])
            df.set_value(i, (key1, key2), arr)
            
diffs = {'ba': [], 'rb': []}

for key in diffs.keys():
    for i in df.index:
        row = df.iloc[i]
        diffs[key].append(row[key]['z_scores'][0])

thresholds = {}

for key in ['ba', 'rb']:
    thresholds[key] = {}
    thresholds[key]['low'] = np.percentile(diffs[key], 15)
    thresholds[key]['high'] = np.percentile(diffs[key], 85)

In [4]:
np.random.seed(10)

nbr_draws = 1000

min_nbr_rats = 5

for to_study in ['H-L', 'H-M', 'M-L']:
    
    size_ = np.infty
    
    for cl in [to_study, to_study[::-1]]:


        subdf = df[((df['ba']['class'] == cl[0]) & (df['rb']['class'] == cl[-1])) & 
                   (df['ba']['nbr_ratings'] >= min_nbr_rats) & (df['rb']['nbr_ratings'] >= min_nbr_rats)]    
        
        if size_ > len(subdf):
            size_ = len(subdf)
            
    nbrs = {'ba': {}, 'rb': {}}

    for cl in [to_study, to_study[::-1]]:

        cls = {'ba': cl[0], 'rb': cl[-1]}

        subdf = df[((df['ba']['class'] == cl[0]) & (df['rb']['class'] == cl[-1])) & 
                   (df['ba']['nbr_ratings'] >= min_nbr_rats) & (df['rb']['nbr_ratings'] >= min_nbr_rats)] 
        subdf.index = range(len(subdf))
        
        subdf = subdf.loc[np.random.choice(len(subdf), size_, replace=False)]
        
        for key in ['ba', 'rb']:

            nbrs[key][cls[key]] = {}

            ranks = np.array([r[min_nbr_rats-1] for r in subdf[key]['z_scores']])

            nbrs[key][cls[key]]['avg'] = np.mean(ranks)

            tmp = []
            # Go through each draw
            for d in range(nbr_draws):

                # Get the indices
                idx = np.random.randint(0, len(subdf[key]), len(subdf[key]))

                ranks_tmp = ranks[idx]

                tmp.append(np.mean(ranks_tmp))
            nbrs[key][cls[key]]['low'] = np.percentile(tmp, 2.5)
            nbrs[key][cls[key]]['high'] = np.percentile(tmp, 97.5)
        

    print('{}:'.format(to_study))
    for key in nbrs:
        print('  {}:'.format(key.upper()))
        for key2 in nbrs[key]:
            print('    {}: {:.3f} [{:.3f}, {:.3f}]'.format(key2, nbrs[key][key2]['avg'], nbrs[key][key2]['low'], nbrs[key][key2]['high']))
    print()

H-L:
  BA:
    H: -0.032 [-0.193, 0.123]
    L: -0.392 [-0.572, -0.231]
  RB:
    L: -0.066 [-0.220, 0.099]
    H: 0.226 [0.045, 0.393]

H-M:
  BA:
    H: 0.270 [0.225, 0.314]
    M: 0.057 [0.006, 0.107]
  RB:
    M: 0.376 [0.334, 0.423]
    H: 0.498 [0.460, 0.542]

M-L:
  BA:
    M: -0.506 [-0.563, -0.450]
    L: -0.657 [-0.721, -0.596]
  RB:
    L: -0.414 [-0.467, -0.363]
    M: -0.229 [-0.279, -0.176]

